# Local Navigation

In [158]:
import tdmclient.notebook
await tdmclient.notebook.start()

In [153]:
%%run_python

#THIS IS THE NON POTENTIAL FIELD METHOD

speed0 = 100       # nominal speed
speedGain = 2      # gain used with ground gradient
obstThrL = 100      # low obstacle threshold to switch state 1->0
obstThrH = 200      # high obstacle threshold to switch state 0->1
obstSpeedGain = 5  # /100 (actual gain: 5/100=0.05)

state = 0          # 0=gradient, 1=obstacle avoidance
obst = [0,0,0,0,0]       # measurements from left and right prox sensors

timer_period[0] = 10   # 10ms sampling time


@onevent 
def timer0():
    global prox_ground_delta, prox_horizontal, motor_left_target, motor_right_target, state, obst, obstThrH, obstThrL, obstSpeedGain, speed0, speedGain 

    obst = [prox_horizontal[0], prox_horizontal[1], prox_horizontal[2], prox_horizontal[3], prox_horizontal[4]]
    if state == 0: #State = 0 -> Global path
        if (obst[0] > obstThrH):
            state = 1
        if (obst[1] > obstThrH):
            state = 1
        if (obst[2] > obstThrH):
            state = 1
        if (obst[3] > obstThrH):
            state = 1
        if (obst[4] > obstThrH):
            state = 1
        if (obst[1] > 10* obstThrH):
            state = 2
        if (obst[2] > 10* obstThrH):
            state = 2
        if (obst[3] > 10* obstThrH):
            state = 2      
    elif state == 1: #State = 1 -> Normal local navigation -> obstacles pop far from the thymio -> normal avoidance
        if obst[0] < obstThrL:
            if obst[1] < obstThrL:
                if obst[2] < obstThrL:
                    if obst[3] < obstThrL:
                        if obst[4] < obstThrL:
                            state = 0
    elif state == 2: # State = 2 -> Rapid local navigation -> obstalces pop close to the thymio -> he go back to go in state 1
        if obst[0] < obstThrL:
            if obst[1] < obstThrL:
                if obst[2] < obstThrL:
                    if obst[3] < obstThrL:
                        if obst[4] < obstThrL:
                            state = 0
                
    if state == 0: # Function to return the thymio in the global path
        leds_top = [0,0,0]
        motor_left_target = speed0 
        motor_right_target = speed0
    if state == 1:
        leds_top = [30,30,30]
        if (obst[0] + obst[1] + obst[2]) > (obst[2] + obst[3] + obst[4]): # Turn on right
            motor_left_target = speed0 + obstSpeedGain * ((obst[0] + obst[1] + obst[2]) // 100)
            motor_right_target = speed0 + obstSpeedGain * ((obst[2] + obst[3] + obst[4]) // 100)
        elif (obst[0] + obst[1] + obst[2]) < (obst[2] + obst[3] + obst[4]): #Turn on left
            motor_left_target = speed0 + obstSpeedGain * ((obst[0] + obst[1] + obst[2]) // 100)
            motor_right_target = speed0 + obstSpeedGain * ((obst[2] + obst[3] + obst[4]) // 100)
        elif (obst[2] > 3 * obstThrH):#Arbitrary direction -> if obstacles in the middle , turn right slightly
                motor_left_target = speed0 + 20*obstSpeedGain
                motor_right_target = speed0 - 20*obstSpeedGain  
    if state == 2:
        motor_left_target = -10*speed0 
        motor_right_target = -10*speed0 
        
                    
        
            
        
            
    
            

In [154]:
%%run_python
motor_right=0
motor_left=0

In [165]:
%%run_python

# THIS IS THE POTENTIAL FIELD METHOD

speed0 = 100      # nominal speed
speedGain = 2     # gain used with ground gradient
obstSpeedGain = [8, 6, -1, -4, -6]
#obstSpeedGain = [6, 4, -2, -6, -8]    # /100
                  # gains used with front proximity sensors 0..4
state = 0
obstThrL = 100      # low obstacle threshold to switch state 1->0
obstThrH = 200      # high obstacle threshold to switch state 0->1
obst = [0,0,0,0,0] 
timer_period[0] = 10  # 10ms sampling time

@onevent 
def timer0():
    global prox_horizontal, speed0, speedGain,obstSpeedGain, motor_left_target, motor_right_target, state, obstThrL, obstThrH
    obst = [prox_horizontal[0], prox_horizontal[1], prox_horizontal[2], prox_horizontal[3], prox_horizontal[4]]
    spLeft = speed0 
    spRight = speed0
    if state == 0: #State = 0 -> Global path
        if (obst[0] > obstThrH):
            state = 1
        if (obst[1] > obstThrH):
            state = 1
        if (obst[2] > obstThrH):
            state = 1
        if (obst[3] > obstThrH):
            state = 1
        if (obst[4] > obstThrH):
            state = 1     
    elif state == 1: #State = 1 -> Normal local navigation -> obstacles pop far from the thymio -> normal avoidance
        if obst[0] < obstThrL:
            if obst[1] < obstThrL:
                if obst[2] < obstThrL:
                    if obst[3] < obstThrL:
                        if obst[4] < obstThrL:
                            state = 0
    if state == 0:
        spLeft = speed0 
        spRight = speed0  
    
    if state == 1: 
        for i in range(5):
            spLeft += obst[i] * obstSpeedGain[i] // 100
            spRight += obst[i] * obstSpeedGain[4 - i] // 100

    motor_left_target = spLeft
    motor_right_target = spRight

In [166]:
%%run_python
motor_right=0
motor_left=0